In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [195]:
ENROLLEES = pd.read_csv('Comma Delimited Data/enrollees.csv').drop(['VERSION'],axis=1)
ENROLLEES = ENROLLEES[['ID','P02HISP','P02SEX', 'P02RACE']]

#### Target Data

In [196]:
dt = pd.read_csv('Comma Delimited Data/outcomes99.csv').drop(['version'],axis=1)
dt.keys()

Index(['id', 'V99RNTCNT', 'V99ERKDATE', 'V99ERKFLDT', 'V99ERKRPCF',
       'V99ERKTLPR', 'V99ERKTPPR', 'V99ERKBLRP', 'V99ERKVSRP', 'V99ERKPODX',
       'V99ERKDAYS', 'V99ERKVSPR', 'V99ERKVSAF', 'V99ERKRPSN', 'V99ERKXRPR',
       'V99ERKXRAF', 'V99ELKDATE', 'V99ELKFLDT', 'V99ELKRPCF', 'V99ELKTLPR',
       'V99ELKTPPR', 'V99ELKBLRP', 'V99ELKVSRP', 'V99ELKPODX', 'V99ELKDAYS',
       'V99ELKVSPR', 'V99ELKVSAF', 'V99ELKRPSN', 'V99ELKXRPR', 'V99ELKXRAF',
       'V99ERHDATE', 'V99ERHFLDT', 'V99ERHRPCF', 'V99ERHPODX', 'V99ERHDAYS',
       'V99ERHVSPR', 'V99ERHVSAF', 'V99ERHRPSN', 'V99ERHXRPR', 'V99ERHXRAF',
       'V99ERHVSRP', 'V99ERHBLRP', 'V99ELHDATE', 'V99ELHFLDT', 'V99ELHRPCF',
       'V99ELHPODX', 'V99ELHDAYS', 'V99ELHVSPR', 'V99ELHVSAF', 'V99ELHRPSN',
       'V99ELHXRPR', 'V99ELHXRAF', 'V99ELHVSRP', 'V99ELHBLRP', 'V99EXLVSQD',
       'V99ERXIOA', 'V99ERXIOAN', 'V99ERXNOA', 'V99ERXNOAN', 'V99ERKLOA',
       'V99ERKLOAN', 'V99ELXIOA', 'V99ELXIOAN', 'V99ELXNOA', 'V99ELXNOAN',
       'V99EL

#### Grouping features names by meaning...

In [197]:
tRPCF = [i for i in dt.columns if i[-4:] =='RPCF']   # Knee and hip replacement status during follow-up
tBLRP = [i for i in dt.columns if i[-4:] =='BLRP']   # Baseline knee or hip replacements
tRPSN = [i for i in dt.columns if i[-4:] =='RPSN']   # Knee or hip replacement seen on follow-up OAI x-ray.
tDAYS = [i for i in dt.columns if i[-4:] =='DAYS']   # Closest OAI contact prior to and after replacement
tVSPR = [i for i in dt.columns if i[-4:] =='VSPR']   # Closest OAI contact prior to replacement
tXRPR = [i for i in dt.columns if i[-4:] =='XRPR']   # Closest OAI visit with x-ray prior to replacement
tXRAF = [i for i in dt.columns if i[-4:] =='XRAF']   # Closest OAI visit with x-ray after the replacement

tVSAF = [i for i in dt.columns if i[-4:] =='VSAF']   # Closest OAI contact after the replacement
tDATE = [i for i in dt.columns if i[-4:] =='DATE']   # Date of a replacement
tTLPR = [i for i in dt.columns if i[-4:] =='TLPR']   # Total or Partial replacement indication
tTPPR = [i for i in dt.columns if i[-4:] =='TPPR']   # Type of Partial replacement indication
tPODX = [i for i in dt.columns if i[-4:] =='PODX']   # Primary pre-operative diagnosis
tDEATH = [i for i in dt.columns if i[-4:] =='EDDCF'] # Death columns

DT_outcome = dt[['id'] + tRPCF + tBLRP + tPODX + tVSPR + tDEATH]

In [198]:
DT_outcome.head()

,id,V99ERKRPCF,V99ELKRPCF,V99ERHRPCF,V99ELHRPCF,V99ERKBLRP,V99ELKBLRP,V99ERHBLRP,V99ELHBLRP,V99ERKPODX,V99ELKPODX,V99ERHPODX,V99ELHPODX,V99ERKVSPR,V99ELKVSPR,V99ERHVSPR,V99ELHVSPR,V99EDDVSPR
0,9000099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9000296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9000622,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,9000798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9001104,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN


#### Grouping features names by affected joint ...

In [199]:
tELK = [i for i in DT_outcome.columns if i[0:6] =='V99ELK']  # Left Knee Columns
tERK = [i for i in DT_outcome.columns if i[0:6] =='V99ERK']  # Right Knee Columns
tELH = [i for i in DT_outcome.columns if i[0:6] =='V99ELH']  # Left Hip Columns
tERH = [i for i in DT_outcome.columns if i[0:6] =='V99ELH']  # Right Hip Columns
DT_outcome[['id'] + tERH].head(5)

,id,V99ELHRPCF,V99ELHBLRP,V99ELHPODX,V99ELHVSPR
0,9000099,NaN,NaN,NaN,NaN
1,9000296,NaN,NaN,NaN,NaN
2,9000622,NaN,0.0,NaN,NaN
3,9000798,NaN,NaN,NaN,NaN
4,9001104,3.0,0.0,4.0,3.0


In [200]:
targets = DT_outcome.fillna(0)
targets.describe()

,id,V99ERKRPCF,V99ELKRPCF,V99ERHRPCF,V99ELHRPCF,V99ERKBLRP,V99ELKBLRP,V99ERHBLRP,V99ELHBLRP,V99ERKPODX,V99ELKPODX,V99ERHPODX,V99ELHPODX,V99ERKVSPR,V99ELKVSPR,V99ERHVSPR,V99ELHVSPR,V99EDDVSPR
count,4.796000e+03,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000
mean,9.513826e+06,0.169725,0.167431,0.072977,0.067973,0.007923,0.005213,0.006047,0.004379,0.065680,0.071309,0.031902,0.035029,0.352585,0.335488,0.139491,0.134696,0.414721
std,2.794781e+05,0.690747,0.686311,0.459963,0.442252,0.088669,0.072018,0.077533,0.066033,0.333187,0.408362,0.263180,0.313783,1.600233,1.534531,1.006929,0.995190,1.813524
min,9.000099e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.283430e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.522042e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.747572e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.999878e+06,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000,8.000000,8.000000,10.000000,10.000000,10.000000,10.000000,12.000000


### Joint Dataset

Reading the data for the joint dataset...

Here, we are trying to consider only the __last visit__, prior to a replacement. We're trying to catch information that refers to a condition before the surgery.

So:
1. Reading datasets from files. Dropping some columns we won't use
2. Renaming columns, changing.. VxxKPNL12, VxxKPNLEVY to... KPNL12, KPNLEVY, etc..   and refering the visit number in a separated column, named 'visit'. (So we can compare variable between the visits)
3. Joining with the information of last visit prior to a replacement.  (this generates a dataset that have one line per visit... per each ID)
4. Keeping just the last visit prior to replacement of each ID.

In [336]:
dt0 = pd.read_csv('Comma Delimited Data/jointsx00.csv').drop(['VERSION'],axis=1)
dt1 = pd.read_csv('Comma Delimited Data/jointsx01.csv').drop(['VERSION'],axis=1) 
dt2 = pd.read_csv('Comma Delimited Data/jointsx02.csv').drop(['VERSION'],axis=1)
dt3 = pd.read_csv('Comma Delimited Data/jointsx03.csv').drop(['VERSION'],axis=1) 
dt4 = pd.read_csv('Comma Delimited Data/jointsx04.csv').drop(['VERSION'],axis=1) 
dt5 = pd.read_csv('Comma Delimited Data/jointsx05.csv').drop(['VERSION'],axis=1) 
dt6 = pd.read_csv('Comma Delimited Data/jointsx06.csv').drop(['VERSION'],axis=1) 
dt7 = pd.read_csv('Comma Delimited Data/jointsx07.csv').drop(['VERSION'],axis=1) 
dt8 = pd.read_csv('Comma Delimited Data/jointsx08.csv').drop(['VERSION'],axis=1) 
dt9 = pd.read_csv('Comma Delimited Data/jointsx09.csv').drop(['VERSION'],axis=1) 
dt10 = pd.read_csv('Comma Delimited Data/jointsx10.csv').drop(['VERSION'],axis=1) 

In [337]:
# Renaming columns (removing Vxx to enable direct comparison between visits)
# the visit number information is kept in the 'visit' column
dts = [dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt8,dt9,dt10]
for j in range(len(dts)):
    d = dts[j]
    col = list(d.columns)
    for i in range(1,len(col)):
        txtcol = col[i]
        col[i] = txtcol[3:]
    d.columns=col
    d['visit'] = j  
JOINT = pd.concat(dts,sort=False)

In [360]:
# getting information about the last contact prior replacement...
DT_outcome['prior_visit'] = DT_outcome[tVSPR].max(axis=1).fillna(0)

In [361]:
JOINT_VISITS = JOINT.merge(DT_outcome[['prior_visit','id']],left_on='ID', right_on='id', how='outer')
JOINT_VISITS.head()

,ID,KPN,KPNREV,KPNREVY,KPNR12,KPNR12M,KPNLEV,KPNLEVY,KPNL12,KPNL12M,...,SF12RP,SF12BP,SF12PF,SF12RE,SF12GH,SF12VT,SF12MH,SF12SF,prior_visit,id
0,9000099,1.0,1.0,2.0,1.0,12.0,1.0,2.0,1.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9000099
1,9000099,NaN,NaN,NaN,1.0,6.0,NaN,NaN,1.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9000099
2,9000099,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9000099
3,9000099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9000099
4,9000099,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9000099


In [362]:
# keeping information of only the last visit
JOINT_VISITS = JOINT_VISITS[JOINT_VISITS['prior_visit'] == JOINT_VISITS['visit']]

In [363]:
JOINT = JOINT_VISITS.drop(['id','visit','prior_visit'],axis=1)

#### MedHist dataset

In [552]:
dt0 = pd.read_csv('Comma Delimited Data/medhist00.csv').drop(['VERSION'],axis=1)
dt1 = pd.read_csv('Comma Delimited Data/medhist01.csv').drop(['VERSION'],axis=1)
dt2 = pd.read_csv('Comma Delimited Data/medhist02.csv').drop(['VERSION'],axis=1) 
dt3 = pd.read_csv('Comma Delimited Data/medhist03.csv').drop(['VERSION'],axis=1)  
dt4 = pd.read_csv('Comma Delimited Data/medhist04.csv').drop(['VERSION'],axis=1) 
dt5 = pd.read_csv('Comma Delimited Data/medhist05.csv').drop(['VERSION'],axis=1)  
dt6 = pd.read_csv('Comma Delimited Data/medhist06.csv').drop(['VERSION'],axis=1)
dt7 = pd.read_csv('Comma Delimited Data/medhist07.csv').drop(['VERSION'],axis=1)  
dt8 = pd.read_csv('Comma Delimited Data/medhist08.csv').drop(['VERSION'],axis=1)
dt9 = pd.read_csv('Comma Delimited Data/medhist09.csv').drop(['VERSION'],axis=1)
dt10 = pd.read_csv('Comma Delimited Data/medhist10.csv').drop(['VERSION'],axis=1)

In [553]:
# Renaming columns (removing Vxx to enable direct comparison between visits)
# the visit number information is kept in the 'visit' column
dts = [dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt8,dt9,dt10]
for j in range(len(dts)):
    d = dts[j]
    col = list(d.columns)
    for i in range(1,len(col)):
        txtcol = col[i]
        col[i] = txtcol[3:]
    d.columns=col
    d['visit'] = j

In [554]:
MEDHIST = pd.concat([dt10,dt9, dt8, dt7, dt6, dt5, dt4, dt3, dt2, dt1],axis=0, join='outer')

In [555]:
####
# Considering drop the columns below

tSREPHR = [i for i in MEDHIST.columns if i[0:4] =='SREP']   # Ever have replacement surgery where all or part of joint was replaced, self-report
tPSDATE = [i for i in MEDHIST.columns if i =='PSDATE']   # Date MRI Safety Screener completed
tSSDATE = [i for i in MEDHIST.columns if i =='SSDATE']   # 

to_drop = tSREPHR + tPSDATE + tSSDATE
MEDHIST = MEDHIST.drop(to_drop,axis=1)

In [556]:
MEDHIST_VISITS = MEDHIST.merge(DT_outcome[['prior_visit','id']],left_on='ID', right_on='id', how='outer')
MEDHIST_VISITS.head()

,ACUSCUR,ACUSCV,ACUSNUM,ACUTCUR,ACUTCV,ACUTNUM,AINJL12,AINJR12,ALZDZ,AMD1012,...,confrec,mrastk3,mritem3,otreasm,remitm,remout,smrielg,visit,prior_visit,id
0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,0.0,9000099
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,0.0,9000099
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.0,9000099
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0,9000099
4,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,0.0,9000099


In [557]:
# keeping information of only the last visit
MEDHIST_VISITS = MEDHIST_VISITS[MEDHIST_VISITS['prior_visit'] == MEDHIST_VISITS['visit']]
MEDHIST = MEDHIST_VISITS.drop(['id','visit','prior_visit'],axis=1)

## ##TODO... Other datasets

In [160]:
'''
Xray datasets do not have one line per ID, so maybe we have to group by...
'''
###TODO
dt0 = pd.read_csv('Comma Delimited Data/xray00.csv').drop(['VERSION'],axis=1)
dt1 = pd.read_csv('Comma Delimited Data/xray01.csv').drop(['VERSION'],axis=1) 
dt2 = pd.read_csv('Comma Delimited Data/xray03.csv').drop(['VERSION'],axis=1) 
dt3 = pd.read_csv('Comma Delimited Data/xray05.csv').drop(['VERSION'],axis=1) 
dt4 = pd.read_csv('Comma Delimited Data/xray06.csv').drop(['VERSION'],axis=1) 
dt5 = pd.read_csv('Comma Delimited Data/xray08.csv').drop(['VERSION'],axis=1)
dt6 = pd.read_csv('Comma Delimited Data/xray10.csv').drop(['VERSION'],axis=1) 


dt0 = pd.read_csv('Comma Delimited Data/acceldatabyday06.csv').drop(['VERSION','V06PAWeekDay','V06VDaySequence','V06PAStudyDay','V06PAMonth'],axis=1)
dt1 = pd.read_csv('Comma Delimited Data/acceldatabyday08.csv').drop(['VERSION','V08PAWeekDay','V08VDaySequence','V08PAStudyDay','V08PAMonth'],axis=1)
dt0 = dt0.groupby('ID').mean()
dt1 = dt1.groupby('ID').mean()


### ##TODO... Medication dataset...

Maybe grouping by ID and INGNAME... aggregating by the mean.
We should generate dummies for INGNAME columns...
Later, we can join to the main training set, by ID

In [161]:
#
dt0 = pd.read_csv('Comma Delimited Data/mif00.csv').drop(['VERSION','V00FRMCODE','V00INGCODE'],axis=1)
dt1 = pd.read_csv('Comma Delimited Data/mif01.csv').drop(['VERSION','V01FRMCODE','V01INGCODE'],axis=1)
dt2 = pd.read_csv('Comma Delimited Data/mif02.csv').drop(['VERSION','V02FRMCODE','V02INGCODE'],axis=1)
dt3 = pd.read_csv('Comma Delimited Data/mif03.csv').drop(['VERSION','V03FRMCODE','V03INGCODE'],axis=1)
dt4 = pd.read_csv('Comma Delimited Data/mif04.csv').drop(['VERSION','V04FRMCODE','V04INGCODE'],axis=1)
dt5 = pd.read_csv('Comma Delimited Data/mif05.csv').drop(['VERSION','V05FRMCODE','V05INGCODE'],axis=1)
dt6 = pd.read_csv('Comma Delimited Data/mif06.csv').drop(['VERSION','V06FRMCODE','V06INGCODE'],axis=1)
dt8 = pd.read_csv('Comma Delimited Data/mif08.csv').drop(['VERSION','V08FRMCODE','V08INGCODE'],axis=1)
dt10 = pd.read_csv('Comma Delimited Data/mif10.csv').drop(['VERSION','V10FRMCODE','V10INGCODE'],axis=1)
#MFI = pd.get_dummies(dt4).groupby('ID').mean()

In [50]:
dts = [dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt8,dt10]
for j in range(len(dts)):
    d = dts[j]
    col = list(d.columns)
    for i in range(1,len(col)):
        txtcol = col[i]
        col[i] = txtcol[3:]
    d.columns=col
    d['visit'] = j
MIF = pd.concat(dts)

In [51]:
MIF = MIF.drop(['visit','MIFNAME'],axis=1).groupby(by=['ID','INGNAME'], as_index=False).mean()
MIF.head()

,ID,INGNAME,MIFFREQ,MIFDUR,MIFUSE
0,9000099,ATENOLOL,2.0,3.0,1.0
1,9000099,METOPROLOL,2.0,3.0,1.0
2,9000099,NAPROXEN,2.0,2.0,1.0
3,9000099,SILDENAFIL,1.0,3.4,1.0
4,9000099,WARFARIN,2.0,3.5,1.0


In [52]:
MIF = pd.get_dummies(MIF)

In [53]:
MIF = MIF.groupby('ID').max()
print(MIF.shape)
MIF.head()

(4536, 931)


,MIFFREQ,MIFDUR,MIFUSE,INGNAME_ABACAVIR,INGNAME_ABIRATERONE ACETATE,INGNAME_ACAMPROSATE,INGNAME_ACARBOSE,INGNAME_ACEBUTOLOL,INGNAME_ACETAMINOPHEN,INGNAME_ACETAZOLAMIDE,...,INGNAME_ZIDOVUDINE,INGNAME_ZILEUTON,INGNAME_ZINC,INGNAME_ZINC OXIDE,INGNAME_ZINC SULFATE,INGNAME_ZIPRASIDONE,INGNAME_ZOLEDRONATE,INGNAME_ZOLMITRIPTAN,INGNAME_ZOLPIDEM,INGNAME_ZONISAMIDE
ID,,,,,,,,,,,,,,,,,,,,,
9000099,2.0,3.500000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9000296,2.0,5.000000,1.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9000622,2.0,5.000000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9001104,2.0,5.000000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9001400,2.0,4.857143,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training and Testing Features

In [571]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

In [572]:
target_name = tRPCF  # choosing replacement adjucation features

# merging targets with jointsx dataset
X = ENROLLEES.merge(DT_outcome[target_name + ['id']], left_on='ID', right_on='id')
X = X.merge(JOINT, on='ID', how='outer')
X = X.merge(MEDHIST,on='ID', how='outer')
#X = X.merge(MIF, how='outer',on='ID')

ids = X['ID']
TARGETS = X[target_name]
X = X.drop(['ID','id'] + target_name,axis=1)

TARGETS['RESULT'] = TARGETS.sum(axis=1)   ## summing all the replacements

In [575]:
train_target = TARGETS['RESULT']

In [576]:
train = X.fillna(-1)
train = pd.get_dummies(train)
train_target = train_target.fillna(0)     # if no information, then 0
train_target[train_target >= 1] = 1       # if greater than 0... at least one replacement occurred

In [577]:
# Create the model with several hyperparameters
model = lgb.LGBMClassifier(objective='binary', boosting_type = 'gbdt', n_estimators = 10000)#, class_weight={0:0.3,1:0.7})

# Initialize an empty array to hold feature importances
feature_importances = np.zeros(train.shape[1])

iterations = 5
# Fit the model twice to avoid overfitting
for i in range(iterations):
    
    # Split into training and validation set
    train_features, valid_features, train_y, valid_y = train_test_split(train, train_target, test_size = 0.25, random_state = i)

    print('\nTraining========================================================== ')        
    # Train using early stopping
    model.fit(train_features, train_y, early_stopping_rounds=100, 
              eval_set = [(valid_features, valid_y)], eval_metric = 'auc', verbose = 200)

    print('\n===========================================================\nModel ')        
    valid_pred = model.predict(valid_features, num_iteration=model.best_iteration_)
    vmetrics = confusion_matrix(valid_y,valid_pred)
    print ('TN',vmetrics[0,0],', FN',vmetrics[1,0],', FP',vmetrics[0,1],', TP',vmetrics[1,1])
    fpr, tpr, thresholds = metrics.roc_curve(valid_y, valid_pred)
    print('AUC = ', metrics.auc(fpr, tpr))    
    print ('Acc:', (vmetrics[0,0] + vmetrics[1,1]) / len(valid_y))
    
    print('==============')
    print('\n0 Frequency: ', (1 - valid_y.sum()/len(valid_y)))
    print(valid_y.describe())

    '''
    RESULT = pd.DataFrame(valid_y)
    RESULT['pred'] = valid_pred
    RESULT['ID'] = ids
    ## IDs with false negative
    print(RESULT[(RESULT['RESULT'] == 1) & (RESULT['pred'] == 0)])
    '''
    
    print('\n==================================\nBASELINE') 
    valid_pred = np.zeros(len(valid_y))
    baseline = confusion_matrix(valid_y,valid_pred)
    fpr, tpr, thresholds = metrics.roc_curve(valid_y, valid_pred)
    print ('TN',baseline[0,0],', FN',baseline[1,0],', FP',baseline[0,1],', TP',baseline[1,1])
    print('AUC = ', metrics.auc(fpr, tpr))
    print ('Acc:', (baseline[0,0] + baseline[1,1]) / len(valid_y))
    
    # Record the feature importances
    feature_importances += model.feature_importances_

feature_importances /= iterations


Training========================================================== 
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.9841
Early stopping, best iteration is:
[102]	valid_0's auc: 0.984724

Model 
TN 1019 , FN 24 , FP 25 , TP 131
AUC =  0.9106074650846621
Acc: 0.9591326105087573

0 Frequency:  0.8707256046705588
count    1199.000000
mean        0.129274
std         0.335643
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: RESULT, dtype: float64

BASELINE
TN 1044 , FN 155 , FP 0 , TP 0
AUC =  0.5
Acc: 0.8707256046705588

Training========================================================== 
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[41]	valid_0's auc: 0.977634

Model 
TN 1025 , FN 19 , FP 26 , TP 129
AUC =  0.9234416385938745
Acc: 0.9624687239366139

0 Frequency:  0.8765638031693077
count    1199.000000
mean        0.123436
std         0.

In [578]:
feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)

In [579]:
# Find the features with zero importance# Find t 
least_important = list(feature_importances[feature_importances['importance'] <= 0]['feature'])
print('There are %d no important features' % len(least_important))

There are 255 no important features


In [570]:
train = train.drop(columns = least_important)
#test = test.drop(columns=least_important)

print('Training shape: ', train.shape)
#print('Testing shape: ', test.shape)

Training shape:  (4796, 366)


## Testing

In [581]:
print(train.shape)

#  Creating test dataset... 60% test... 40% train..
train_, test, train_target_, test_target = train_test_split(train, train_target, test_size = 0.6)
print(train_.shape, test_target.shape)

iterations = 10
# Fit the model twice to avoid overfitting
for i in range(iterations):

    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'gbdt', n_estimators = 10000, class_weight={0:0.3,1:0.7})    
    
    train_features, valid_features, train_y, valid_y = train_test_split(train_, train_target_, test_size = 0.25, random_state = i)
    model.fit(train, train_target, early_stopping_rounds=100, 
                  eval_set = [(valid_features, valid_y)], eval_metric = 'auc', verbose = 200)


    print('==== Testing best iteration ====')
    test_pred = model.predict(test, num_iteration=model.best_iteration_)

        
    vmetrics = confusion_matrix(test_target,test_pred)
    print ('TN',vmetrics[0,0],', FN',vmetrics[1,0],', FP',vmetrics[0,1],', TP',vmetrics[1,1])
    fpr, tpr, thresholds = metrics.roc_curve(test_target, test_pred)
    print('AUC = ', metrics.auc(fpr, tpr))    
    print ('Acc:', (vmetrics[0,0] + vmetrics[1,1]) / len(test_target))

(4796, 621)
(1918, 621) (2878,)
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 1
Early stopping, best iteration is:
[112]	valid_0's auc: 1
==== Testing best iteration ====
TN 2523 , FN 1 , FP 2 , TP 352
AUC =  0.9981875298011387
Acc: 0.9989576094510076
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[52]	valid_0's auc: 1
==== Testing best iteration ====
TN 2515 , FN 3 , FP 10 , TP 350
AUC =  0.9937705101954954
Acc: 0.9954829742876998
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[95]	valid_0's auc: 0.999984
==== Testing best iteration ====
TN 2525 , FN 2 , FP 0 , TP 351
AUC =  0.9971671388101984
Acc: 0.9993050729673384
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[95]	valid_0's auc: 0.999981
==== Testing best iteration ====
TN 2525 , FN 2 , FP 0 , TP 351
AUC =  0.9971671388101984
Acc: 0.9993050729